In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import copy
import random as rd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [52]:
#there two types of impurity here, entropy and information theory
#of course you can use miscalculation error
#i just don’t find it very convincing, thats all
def impurity(data,method):

    output=0.0
    
    for i in set(data):      
                    
        prob=list(data).count(i)/len(data)
        
        if method=='gini':
            output+=prob**2
        
        #entropy
        else:
            output+=prob*(np.log(prob)/np.log(2))
    
    extra=1 if method=='gini' else 0
    
    return extra-output

In [53]:
#information gain is function built upon impurity
def information_gain(parent,left,right,method):
    
    leftc=len(left)/len(parent)
    rightc=len(right)/len(parent)
    
    sumc=leftc*impurity(left,method)+rightc*impurity(right,method)
    
    return impurity(parent,method)-sumc

In [54]:
#we would love to select a criterion that maximizes information gain
#in human language, we wanna have two chaotic children nodes after the split
#when the child node shows non dominant class within the group
#it has the largest entropy (uncertainty) which maximizes information gain
def node_split(df,x,y,method):
    
    info=0.0
    target=0.0
    
    for i in df[x].tolist():
        
        #values smaller than the threshold go to left child node
        left=df[y].loc[df[x]<i].tolist()
        right=df[y].loc[df[x]>=i].tolist()
        
        temp=information_gain(df[y].tolist(),left,right,method)
            
        if info<temp:
            info=temp
            target=i
    
    #we output both split threshold and its information gain
    return target,info

In [55]:
#this function examines all the potential split threshold
#we can split from the current input factor or use another input factor
#at the end, we only select the criterion that has the largest information gain
def factors_split(data,y,method,random=False):
    
    output=pd.DataFrame()
    
    #random argument is used for random forest
    #unlike conventional decision tree
    #we dont compare information gain from all the input features
    #we randomly select a couple of features for every split
    #and compare the information gain of these randomly selected features
    if random==True:
        cols=[i for i in data.columns if i!=y]
        bagging=rd.sample(cols,int(len(cols)**0.5))
        bagging.append(y)
        df=copy.deepcopy(data)[bagging]
    else:
        df=copy.deepcopy(data)
    
    for i in df.columns:
        
        if i!=y:

            target,info=node_split(df,i,y,method)
            output[i]=[target,info]
        
    output=output.T
    output.reset_index(inplace=True)
    output.columns=['colname','target','info']
    
    #sort by information gain in descending order
    output.sort_values(by='info',inplace=True,
                       ascending=False)
    output.reset_index(inplace=True,drop=True)
    
    return output

In [56]:
#like graph theory, data structure plays a bigger role than algorithm
#how to build our tree properly is crucial to our model
#tree building is done recursively
#if you are not confident with your knowledge in recursion
# https://github.com/je-suis-tm/recursion-and-dynamic-programming
def build_tree(df,parentname,tree,xcol,ycol,
               max_tree_depth=10,min_node_num=2,
               method='gini',**kwargs):

    temp=df.loc[tree.nodes[parentname]['data']]
    
    #this determines how to split to have the largest information gain
    guide=factors_split(temp,ycol,method,**kwargs)
    
    target=guide['target'][0]
    colname=guide['colname'][0]
    
    left=temp.loc[temp[colname]<target].index.tolist()
    right=temp.loc[temp[colname]>=target].index.tolist()
    
    #here are the conditions to build children nodes
    
    #we need to preset a maximum tree depth
    #the deeper the tree is, the more likely it will be overfitted
    #one of the biggest flaws of decision tree is overfit
    #we can apply pruning to clean up the tree after we get the model
    #when we reach the maximum tree depth
    #we call it an end to splitting
    #we take the majority within the group as classification
    if len(parentname)>=max_tree_depth: 
        ind=tree.nodes[parentname]['data']
        nx.set_node_attributes(tree,
                               {parentname:
                                {'label':max(df[ycol].loc[ind].tolist(),
                                             key=df[ycol].loc[ind].tolist().count)}})
        
        return

    #there could be a case where the biggest information gain leaves all elements to one child node
    #the other child node has nothing at all
    #we should immediately stop splitting and pick the majority as the classification
    if not left or not right:

        ind=tree.nodes[parentname]['data']
        nx.set_node_attributes(tree,{parentname:
                                     {'label':max(df[ycol].loc[ind].tolist(),
                                                  key=df[ycol].loc[ind].tolist().count)}})
        
        return
    
    #if the data inside the child node has homogenous classification
    #there is no need for further splitting
    #but we wont return the function here cuz this is only the case for left child node
    if len(df[ycol].loc[left].unique())==1:
        
        #when we create a left child node, we just add L to its node name
        #we can track the depth of current node easily by this naming rule
        tree.add_node(f'{parentname}L',data=left,
                      target=target,colname=colname,
                      label=df[ycol].loc[left].iloc[0]
                     )

        tree.add_edge(f'{parentname}', f'{parentname}L')


    else:
        
        #there is another preset parameter, minimum data inside the node
        #if we have too few samples inside the node
        #our split becomes meaningless
        #if we have enough samples inside the node
        #we continue splitting and call our function recursively
        if len(left)>min_node_num:
            tree.add_node(f'{parentname}L',data=left,
                          target=target,colname=colname
                         )

            tree.add_edge(f'{parentname}', f'{parentname}L')
            
            build_tree(df,f'{parentname}L',tree,
                       xcol,ycol,method=method,**kwargs)

        else:
            tree.add_node(f'{parentname}L',data=left,
                          target=target,colname=colname,
                          label=max(df[ycol].loc[left].tolist(),
                                    key=df[ycol].loc[left].tolist().count)
                         )

            tree.add_edge(f'{parentname}', f'{parentname}L')


    #the right child node has the same case as the left one
    if len(df[ycol].loc[right].unique())==1:
        tree.add_node(f'{parentname}R',data=right,
                      target=target,colname=colname,
                      label=df[ycol].loc[right].iloc[0]
                     )

        tree.add_edge(f'{parentname}', f'{parentname}R')

    else:

        if len(right)>min_node_num:
            tree.add_node(f'{parentname}R',data=right,
                          target=target,colname=colname
                         )
            tree.add_edge(f'{parentname}', f'{parentname}R')
            
            build_tree(df,f'{parentname}R',tree,
                       xcol,ycol,method=method,**kwargs)

        else:
            tree.add_node(f'{parentname}R',data=left,
                          target=target,colname=colname,
                          label=max(df[ycol].loc[left].tolist(),
                                    key=df[ycol].loc[left].tolist().count)
                         )

            tree.add_edge(f'{parentname}', f'{parentname}R')

In [57]:
#once we have our tree structure built, prediction is simple
#it is kinda like traversal algorithm in graph theory
#it travels down the tree structure like depth first search
# https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb
#the function itself is recursive as well
#but when we reach the end of the tree, we do not go one level up
#we get the label attribute of the node and exit
def predict(tree,nodename,data,ind):

    colname=tree.nodes[nodename]['colname']
    target=tree.nodes[nodename]['target']
    edges=[i[1] for i in tree.edges(nodename)]
    
    if not edges:
        forecast=tree.nodes[nodename]['label']
        return forecast
    
    if data[colname].iloc[ind]<target:
        nodename=edges[0]
    else:
        nodename=edges[1]
        
    forecast=predict(tree,nodename,data,ind)
    
    return forecast

In [58]:
#this is a self-made function to create coordinates for a perfect binary tree
#graph visualization in networkx is problematic
#random positions do not fit for a perfect binary tree
#the function borrows an idea from how to plot a pascal triangle
# https://github.com/je-suis-tm/recursion-and-dynamic-programming/blob/master/pascal%20triangle%20better%20structured.py
#though no recursion is implemented

#xwid refers to the horizontal distance between sibling nodes
#ywid refers to the vertical distance between parent node and children nodes
def binary_tree_pos(tree,xwid=5,ywid=-7):    
    
    #this is the depth of the tree
    #basically finding the maximum length of node names
    rownum=max([len(i) for i in tree.nodes])

    lastrow=[i*xwid for i in range(2**(rownum-1))]

    xmat=[lastrow]
    
    #we start from the deepest level to calculate coordinates for each node
    for i in range(rownum-1,0,-1):
        row=[]
        for j in range(1,len(xmat[-1]),2):
            
            #since we assume this is a perfect binary tree
            #the parent node should be located right between two children nodes
            row.append(np.mean([xmat[-1][j],xmat[-1][j-1]]))
        xmat.append(row)

    ymat=[]
    
    #get the correct order
    xmat=xmat[::-1]
    
    #y coordinate simply depends on the depth of a node
    ymat=[[i*ywid for j in xmat[i]] for i in range(len(xmat))]
    
    root='0'
    nodes=[root]
    
    #using our naming rule to get a whole set of node names
    for i in range(1,rownum):
        row=[]
        for j in nodes[-1]:
            row.append(j+'L')
            row.append(j+'R')

        nodes.append(row)
    
    #map coordinates into the specific node
    #even though decision tree is no perfect binary tree
    #some of left/right do not exist
    #as we dont create edges from parent node to non-existent child
    #these non-existent nodes wont appear in visualization
    pos={}
    for row in range(len(nodes)):
        for column in range(len(nodes[row])):
            pos[nodes[row][column]]=(xmat[row][column],
                                     ymat[row][column])
            
    return pos

In [59]:
#concatenate all the functions
def decision_tree(train,test,xcol,ycol,method='gini'):   
    
    #decision tree is a directed graph structure
    #we can only go down but not up
    tree=nx.DiGraph()

    guide=factors_split(train,ycol,method)
    target=guide['target'][0]
    colname=guide['colname'][0]
    
    #the root node has to be added outside of build_tree function
    tree.add_node('0',data=train.index.tolist(),
                 target=target,colname=colname)

    parentname='0'

    build_tree(train,parentname,tree,
               xcol,ycol,method=method)
    
    #visualization using networkx
    ax=plt.figure(dpi=500)
    nx.draw(tree,with_labels=True,
            node_size=100,pos=binary_tree_pos(tree),
            font_size=5)
    
    forecast=[]    
    for i in train.index.tolist():

        forecast.append(predict(tree,'0',train,i))

    train['projection']=forecast
    
    forecast=[]
    for i in test.index.tolist():

        forecast.append(predict(tree,'0',test,i))

    test['projection']=forecast
    
    print('\ntrain accuracy: %s'%(len(train[train['y']==train['projection']])/len(train)))
    print('\ntest accuracy: %s'%(len(test[test['y']==test['projection']])/len(test)))

In [100]:
def random_forest(train,test,xcol,ycol,
                  method='gini',treenum=10):   
    
    
    oob=[]
    training_forecast=[]
    testing_forecast=[]
    oob_forecast=[]
    testing=copy.deepcopy(test)
    
    for i in range(treenum):
        
        #bagging, bootstrap aggregating
        #we randomly select data from the original dataset
        #to form a new subset with the same size of the original
        #in the new subset, data points are not unique
        #some of the data points can occur more than once
        #some of the data points from the original may not appear
        #we call these outliers out-of-bag
        #we can use out-of-bag errors to validate the model
        #usually out-of-bag set is approximately 36% of the original
        #as for why, you can read it from the link below
        #https://towardsdatascience.com/what-is-out-of-bag-oob-score-in-random-forest-a7fa23d710
        training=copy.deepcopy(train)
        
        ind=[rd.choice(training.index.tolist()) for j in range(len(training))]
        oob=[i for i in range(len(training)) if i not in ind]
        
        training=training.loc[ind]
        training.reset_index(inplace=True,drop=True)

        tree=nx.DiGraph()

        guide=factors_split(training,ycol,method)
        target=guide['target'][0]
        colname=guide['colname'][0]

        tree.add_node('0',data=training.index.tolist(),
                     target=target,colname=colname)

        parentname='0'

        build_tree(training,parentname,tree,
                   xcol,ycol,method=method,random=True)

        #there will be at least ten charts
        #so we skip the visualization

        #in random forest, there are way too many decision trees
        #and each tree may deliver a different classification
        #we need to count the vote to get the majority as the prediction
        for j in train.index.tolist():
            training_forecast.append(predict(tree,'0',train,j))
        for j in testing.index.tolist():
            testing_forecast.append(predict(tree,'0',testing,j))
            

        #for each subset we should have one out-of-bag set
        #we treat each out-of-bag set as a testing set
        #besides traditional train test split
        #out-of-bag can be regarded as cross validation here
        #if u wanna know why, you can read the paper from this url
        # https://www.stat.berkeley.edu/~breiman/OOBestimation.pdf
        #we calculate oob score for each subset then take the average
        train_oob=train.loc[oob]
        train_oob.reset_index(inplace=True,drop=True) 
        forecast=[]
        for j in train_oob.index.tolist():
            forecast.append(predict(tree,'0',train_oob,j))
        train_oob['projection']=forecast
        oob_forecast.append(
            len(
                train_oob[
                    train_oob['price 2']==train_oob['projection']])/len(train_oob))
        

    
    forecast=[]
    for i in range(len(train)):
        temp=training_forecast[i::len(train)]
        forecast.append(max(set(temp),key=temp.count) )
    train['projection']=forecast
    
    
    forecast=[]
    for i in range(len(testing)):
        temp=testing_forecast[i::len(testing)]
        forecast.append(max(set(temp),key=temp.count) )
    testing['projection']=forecast
    

    
    print('\ntrain accuracy: %s'%(
        len(train[train['price 2']==train['projection']])/len(train)))
    print('\ntest accuracy: %s'%(
        len(testing[testing['price 2']==testing['projection']])/len(testing)))
    print('\noob accuracy: %s'%(np.mean(oob_forecast)))

In [61]:
import sys
import os
searchPath=os.path.abspath('..')
sys.path.append(searchPath)
import os  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline
from sklearn.model_selection import  train_test_split

In [62]:
# Loading the dataset in python with the name df and displaying first 20 rows: 
df=pd.read_csv("2008.csv",sep = ';')
df.head(20)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1
5,2008,4,1,6,29,1,3,C56,6,1,2,57,1,4
6,2008,4,1,7,29,1,3,C57,5,1,2,33,2,4
7,2008,4,1,8,29,1,4,P67,9,5,1,38,1,4
8,2008,4,1,9,29,1,4,P82,6,4,2,48,1,5
9,2008,4,1,1,29,2,2,B31,9,5,1,57,1,2


In [76]:
df=df[0:1000]

In [77]:
test=df['price 2']

In [78]:
train=df[df.columns.drop('price 2')]
train=train[train.columns.drop('page 2 (clothing model)')]
train=train[train.columns.drop('month')]
train=train[train.columns.drop('day')]
train=train[train.columns.drop('year')]

In [79]:
train.shape

(1000, 9)

In [80]:
train=train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [81]:
train

,order,country,session ID,page 1 (main category),colour,location,model photography,price,page
0,0.000,0.571429,0.0,0.000000,0.000000,0.8,0.0,0.156250,0.00
1,0.025,0.571429,0.0,0.000000,0.000000,1.0,0.0,0.234375,0.00
2,0.050,0.571429,0.0,0.333333,0.692308,0.2,0.0,0.531250,0.00
3,0.075,0.571429,0.0,0.333333,0.384615,1.0,1.0,0.312500,0.00
4,0.100,0.571429,0.0,0.333333,0.230769,0.4,1.0,0.531250,0.00
...,...,...,...,...,...,...,...,...,...
995,0.175,0.714286,1.0,1.000000,0.230769,0.8,1.0,0.312500,0.00
996,0.200,0.714286,1.0,0.666667,1.000000,0.6,0.0,0.390625,0.00
997,0.225,0.714286,1.0,0.666667,1.000000,1.0,0.0,0.609375,0.50
998,0.250,0.714286,1.0,0.666667,1.000000,1.0,0.0,0.609375,0.50


In [88]:
train=pd.concat([train,test],axis=1)#[ ]容易忘掉

In [83]:
# # data
# def create_data():
# #     df['label'] = iris.target
# #     df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
#     data = train.iloc[:1000,:]

# #     for i in range(len(data)):
# #         if data[i,-1] == 2:
# #             data[i,-1] = -1
#     print(data)
#     return data[:,[0,1,2,3,4,5,6,7,8]], data[:,-1]

In [84]:
# X, y = create_data()

In [90]:
X_train, X_test, y_train, y_test = train_test_split(train,test, test_size=0.25)

In [91]:
X_train

,order,country,session ID,page 1 (main category),colour,location,model photography,price,page,price 2
221,0.025,0.571429,0.209581,1.000000,0.461538,1.0,0.0,0.234375,0.00,2
736,0.025,0.571429,0.730539,1.000000,0.153846,0.0,0.0,0.156250,0.00,2
282,0.000,0.000000,0.269461,0.000000,0.153846,0.6,0.0,0.687500,0.00,1
297,0.150,0.000000,0.287425,0.666667,0.923077,0.4,0.0,0.468750,0.00,1
414,0.300,0.428571,0.377246,0.333333,0.076923,0.6,0.0,0.312500,0.00,2
...,...,...,...,...,...,...,...,...,...,...
245,0.000,0.000000,0.251497,0.333333,0.923077,0.0,0.0,0.312500,0.25,2
413,0.275,0.428571,0.377246,0.333333,0.615385,0.6,0.0,0.609375,0.25,1
803,0.125,0.571429,0.772455,1.000000,1.000000,0.6,0.0,0.156250,0.50,2
932,0.200,0.571429,0.934132,0.666667,0.846154,0.8,0.0,0.234375,0.25,2


In [92]:
xcol=['order','country','session ID','page 1 (main category)','colour','location','model photography','price','page']

In [93]:
ycol='price 2'

In [101]:
random_forest(train,train,xcol,ycol)


train accuracy: 0.744

test accuracy: 0.744

oob accuracy: 0.6143377641529909
